# 1. Load packages

In [ ]:
from analysis_tools.common import *
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

# 2. Load dataset

In [ ]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

with ProgressBar():
    X_train_full = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.train)])
    X_test       = compute(*[delayed(cv2.imread)(path) for path in ls_file(PATH.test)])
y_train_full = train_full_data_meta[['label']]
    
print("- Number of train full data:", len(X_train_full))
print("- Number of test data:", len(X_test))

# 3. Training

In [ ]:
from sklearn.preprocessing import OneHotEncoder

IMG_SIZE    = 512
input_shape = (IMG_SIZE, IMG_SIZE, 3)

with ProgressBar():
    X_train_full = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_train_full]))
    X_test       = np.array(compute(*[delayed(cv2.resize)(X, [IMG_SIZE, IMG_SIZE]) for X in X_test]))
y_enc = OneHotEncoder(sparse=False, dtype=bool)
y_train_full = y_enc.fit_transform(y_train_full)
n_classes = len(y_enc.categories_[0])
print("- Number of classes:", n_classes)

In [ ]:
aug_model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

def preprocess(ds, training, batch_size, augment=True):
    ds = ds.cache().batch(batch_size)
    if training:
        ds = ds.shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)
        if augment:
            ds = ds.map(lambda X, y, sw: (aug_model(X), y, sw), num_parallel_calls=tf.data.AUTOTUNE)
    return ds

# fig, axes = plt.subplots(5, 15, figsize=(30, 10))
# for row, ax_cols in enumerate(axes):
#     for ax in ax_cols:
#         ax.imshow(aug_model(X_train_full[row]))
#         ax.axis('off')
# plt.show()

## 3.1 Fix pretrained model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, stratify=y_train_full)
sample_weight_train = compute_sample_weight(class_weight='balanced', y=y_train.argmax(1))
sample_weight_val   = compute_sample_weight(class_weight='balanced', y=y_val.argmax(1))

train_ds = preprocess(tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weight_train)), True, BATCH_SIZE)
val_ds   = preprocess(tf.data.Dataset.from_tensor_slices((X_val, y_val, sample_weight_val)), False, BATCH_SIZE)
test_ds  = preprocess(tf.data.Dataset.from_tensor_slices(X_test), False, BATCH_SIZE)

print("- train.shape:", X_train.shape, y_train.shape)
print("- val.shape:", X_val.shape, y_val.shape)
print("- test.shape:", X_test.shape)

In [ ]:
from tensorflow_addons.metrics import F1Score

def build_model(n_classes, strategy):
    with strategy.scope():
        base_model = keras.applications.EfficientNetB4(include_top=False, input_shape=input_shape)
        base_model.trainable = False

        inputs  = keras.Input(input_shape)
        hidden  = base_model(inputs, training=False)
        hidden  = keras.layers.GlobalAveragePooling2D()(hidden)
        outputs = keras.layers.Dense(n_classes, activation='softmax')(hidden)
        model   = keras.Model(inputs, outputs)
        
        model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
    return model, base_model

In [ ]:
from analysis_tools.modeling import *

model_name = 'fix_pretrained_model_nadam_lr_scheduler_b4'
model, base_model = build_model(n_classes, strategy)
epochs = 2
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=get_callbacks(patience=50, plot_path=join(PATH.result, 'proposed3', model_name), init_lr=2e-3, epochs=epochs, warmup_epoch=30, min_lr=1e-3))

## 3.2 Fine-tuning

In [ ]:
model_name = 'fine_tuning_nadam_lr_scheduler_b5'

with strategy.scope():
    base_model.trainable = True
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=[F1Score(num_classes=n_classes, average='macro')])
epochs = 200
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=get_callbacks(patience=50, plot_path=join(PATH.result, 'proposed3', model_name), init_lr=2e-4, epochs=epochs, warmup_epoch=30, min_lr=1e-4))

In [ ]:
as

# 4. Evaluation

In [ ]:
submission_file_path = join(PATH.output, f'proposed3_{model_name}.csv')

pred_test = model.predict(test_ds)
submission = pd.read_csv(join(PATH.input, 'sample_submission.csv'), index_col=0)
submission['label'] = y_enc.inverse_transform(pred_test)
submission.to_csv(submission_file_path)
submission

# 5. Submission

In [ ]:
from dacon_submit_api.dacon_submit_api import post_submission_file

result = post_submission_file(
    submission_file_path,
    '137ff236e305f302819b930b3b5b72e948603f23c5249a516c32b536d5187a03', 
    '235894', 
    '어스름한 금요일 밤에', 
    f'proposed3_{model_name}'
)